# Test OpenAI Agents SDK
- Implement a workflow to write a daily AI newsletter

In [1]:
import os
import yaml
import dotenv
import logging
import json
import yaml
from datetime import datetime
import time
import random
import glob
import pickle
import sqlite3

from pathlib import Path

import asyncio
import nest_asyncio

import pydantic
from pydantic import BaseModel, Field, RootModel
from typing import Dict, TypedDict, Type, List, Optional, Any, Iterable, Text
from dataclasses import dataclass, field
from enum import Enum

import numpy as np
import pandas as pd

import pandas as pd
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import hdbscan

import openai
from openai import AsyncOpenAI

import agents
from agents.exceptions import InputGuardrailTripwireTriggered
from agents import (Agent, Runner, Tool, OpenAIResponsesModel, 
                    ModelSettings, FunctionTool, InputGuardrail, GuardrailFunctionOutput,
                    SQLiteSession, set_default_openai_api, set_default_openai_client
                   )


import tenacity
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

from IPython.display import HTML, Image, Markdown, display

from log_handler import SQLiteLogHandler, setup_sqlite_logging, sanitize_error_for_logging
from config import LOGDB
from llm import LLMagent, LangfuseClient  # methods to apply prompts async to large batches
from db import Url 

from fetch import Fetcher # fetch news urls
from newsletter_state import NewsletterAgentState, StepStatus
from news_agent import NewsletterAgent


In [2]:
print(f"OpenAI:            {openai.__version__}")
print(f"OpenAI Agents SDK  {agents.__version__}")
print(f"Pydantic           {pydantic.__version__}")


OpenAI:            1.109.0
OpenAI Agents SDK  0.3.1
Pydantic           2.11.9


In [3]:
dotenv.load_dotenv()

# to run async in jupyter notebook
nest_asyncio.apply()

# verbose OpenAI console logging if something doesn't work
# logging.basicConfig(level=logging.DEBUG)
# openai_logger = logging.getLogger("openai")
# openai_logger.setLevel(logging.DEBUG)


In [4]:
# modules create a default logger, or we can pass this logger

def setup_logging(session_id: str = "default", db_path: str = "agent_logs.db") -> logging.Logger:
    """Set up logging to console and SQLite database."""

    # Create logger
    logging.basicConfig(level=logging.INFO)

    logger = logging.getLogger(f"NewsletterAgent.{session_id}")
    logger.setLevel(logging.INFO)

    # Clear any existing handlers
    logger.handlers.clear()

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    console_formatter = logging.Formatter(
        '%(asctime)s | %(name)s | %(levelname)s | %(message)s',
        datefmt='%H:%M:%S'
    )
    console_handler.setFormatter(console_formatter)

    # SQLite handler
    sqlite_handler = SQLiteLogHandler(db_path)
    sqlite_handler.setLevel(logging.INFO)
    sqlite_formatter = logging.Formatter('%(message)s')
    sqlite_handler.setFormatter(sqlite_formatter)

    # Add handlers to logger
    logger.addHandler(console_handler)
    logger.addHandler(sqlite_handler)

    # Prevent propagation to root logger
    logger.propagate = False

    return logger

logger = setup_logging("newsletter_agent", "test_logs.db")

# Log some test messages
logger.info("Test info message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.warning("Test warning message", extra={
    'step_name': 'test_step',
    'agent_session': 'demo_session'
})

logger.error("Test error message", extra={
    'step_name': 'error_step',
    'agent_session': 'demo_session'
})

sanitize_error_for_logging("log with some bad stuff for the filter: sk-proj-123456789012345678901234567890123456789012345678")

22:46:56 | NewsletterAgent.newsletter_agent | INFO | Test info message
22:46:56 | NewsletterAgent.newsletter_agent | WARNING | Test warning message
22:46:56 | NewsletterAgent.newsletter_agent | ERROR | Test error message


'log with some bad stuff for the filter: [API_KEY_REDACTED]'

# Run Agent Worfklow

In [5]:
print("🚀 Creating NewsletterAgent...")

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

# Set up OpenAI client for the agents SDK
set_default_openai_client(AsyncOpenAI(api_key=api_key))

# set up state
session_id = 'test_newsletter_20250926123206126884'
step_name = 'step_05_cluster_by_topic'
# del session_id

do_download=False
process_since='2025-09-24 18:00:00'

# Create agent with persistent state
if 'session_id' in vars():
    # load state from db for session_id and state
    print("session_id is defined")
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db", 
                                 do_download=do_download,
                                 process_since=process_since)
    state = state.load_from_db(step_name)
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=True, timeout=30)    
else:
    # create new session
    print("session_id is not defined")
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S%f")    
    session_id = f"test_newsletter_{timestamp}"
    print(session_id)
    state = NewsletterAgentState(session_id=session_id, 
                                 db_path="newsletter_agent.db",
                                 do_download=do_download,
                                 process_since=process_since
                                ) 
    agent = NewsletterAgent(session_id=session_id, state=state, verbose=False, timeout=30)
    state.serialize_to_db("initialize")

22:47:04 | NewsletterAgent.test_newsletter_20250926123206126884 | INFO | Using provided state with 323 articles


🚀 Creating NewsletterAgent...
session_id is defined
test_newsletter_20250926123206126884
Initialized NewsletterAgent with persistent state and 9-step workflow
Session ID: test_newsletter_20250926123206126884


In [6]:
state.get_status()

{'headlines': {'total': 323},
 'sources': {'config_file': 'sources.yaml', 'loaded_sources': 0},
 'topics': {'cluster_topics': 0, 'topics': []},
 'workflow': {'current_step': 'step_06_rate_articles',
  'workflow_complete': False,
  'workflow_status': 'started',
  'workflow_status_message': '',
  'progress_percentage': 55.55555555555556,
  'max_edits': 2,
  'concurrency': 16},
 'processing': {'topic_clusters': 0,
  'newsletter_sections': 0,
  'final_newsletter_length': 0}}

In [ ]:
state.get_current_step()


In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run a workflow step
user_prompt = "Run step 1, fetch urls"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


In [ ]:
pd.DataFrame(state.headline_data) 


In [ ]:
countdf = pd.DataFrame(state.headline_data) \
    .groupby("source") \
    .count()[["id"]] \
    .reset_index() \
    .rename(columns={'id': 'count'}) \
    .sort_values("count", ascending=False)
countdf 


In [ ]:
# Run tool directly without LLM processing an input prompt or results
# user_prompt = "Run step 2, filter urls"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("filter_urls")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 3, download full articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_tool_direct("download_articles")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 4, Summarize articles"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("extract_summaries")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
# User prompt to run workflow
# user_prompt = "Run step 5, Cluster articles by topic"
# print(f"\n📝 User prompt: '{user_prompt}'")
# print("=" * 80)

start_time = time.time()
result = await agent.run_tool_direct("cluster_by_topic")
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)


In [ ]:
headline_df

In [ ]:
# User prompt to run workflow
user_prompt = "Show the workflow status"

print(f"\n📝 User prompt: '{user_prompt}'")
print("=" * 80)

# Run the agent with persistent state
start_time = time.time()
result = await agent.run_step(user_prompt)
duration = time.time() - start_time

print("=" * 80)
print(f"⏱️  Total execution time: {duration:.2f}s")
print(f"📊 Final result:")
print(result)

In [ ]:
from newsletter_state import StepStatus
agent.state.step_05_cluster_by_topic = StepStatus.NOT_STARTED


- rate articles
- load sources into db with reputation
- get domain from each url and put in headline_df
- look up source and reputation
- prompt for on topic , important, high quality
- run bradley terry
- combine ratings

    aidf['rating'] = aidf['reputation'] \
        + aidf['adjusted_len'] \
        + aidf['on_topic'] \
        + aidf['importance'] \
        - aidf['low_quality'] \
        + aidf['bt_z'] \
        + aidf['recency_score']

set selected flag using rating
store to db

- next steps select sections
- clean clusters , combine clusters, select sections 

In [ ]:
state = await agent.run_step("get state")
state 


In [ ]:
inspect_result = await agent.run_step("inspect state")


In [ ]:
state = await agent.get_state_direct()


In [ ]:
print(status_result)


In [ ]:
from importlib import reload

In [ ]:
import sys
import dotenv
dotenv.load_dotenv()

# Delete the module from cache
if 'llm' in sys.modules:
    print("llm")
    del sys.modules['llm']

# Now you can import it again
import llm
from llm import LLMagent


In [ ]:
agent = LLMagent(
  system_prompt="Classify as funny or not funny. Return only 1 for funny, 0 for not funny",
  user_prompt="Text: {text}\n",
  output_type=str,  # Not used for logprobs
  model="gpt-4.1-mini"
)

In [ ]:
_, logprobs = await agent.prompt_dict_chat_probs({'text': 'take my wife. please.'})
logprobs 



In [ ]:
logprobs.content[0].logprob



In [ ]:
if not logprobs or getattr(logprobs, 'content', None) is None:
            raise ValueError("Invalid logprobs_data. Must contain 'content' key with non-None value.")


In [ ]:
        first_token_logprobs = logprobs.content[0]


In [ ]:
first_token_logprobs.top_logprobs


In [ ]:
top_logprobs = first_token_logprobs.top_logprobs
top_logprobs

In [ ]:
agent._extract_token_probabilities(logprobs, "1") 


In [ ]:
await agent.run_prompt_with_probs(text="fruit flies like a banana")

In [ ]:
import pandas as pd

df = pd.DataFrame({
      "text": [
          "fruit flies like a banana",
          "yo momma so low she plays squash against the curb",
          "thou shalt not kill",
          "first came the thunder, then came the rain"
      ]
  })

In [ ]:
df 


In [ ]:
await agent.filter_dataframe(
      df[["text"]],
      return_probabilities=True,
      target_tokens=["1"]
  )


In [7]:
state.headline_df

,id,source,title,url,published,rss_summary,isAI,status,final_url,html_path,last_updated,text_path,content_length,summary,description,topics
0,191,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,True,success,https://arstechnica.com/ai/2025/09/chatgpt-pul...,download/html/ChatGPT_Pulse_delivers_morning_u...,2025-09-25T20:30:52Z,download/text/ChatGPT_Pulse_delivers_morning_u...,2521,"- OpenAI launched ChatGPT Pulse, a new mobile ...",New mobile chatbot feature analyzes conversati...,"[ChatGPT Pulse, Personalization, Asynchronous ..."
1,182,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,True,success,https://arstechnica.com/information-technology...,download/html/Experts_urge_caution_about_using...,2025-09-25T18:10:50Z,download/text/Experts_urge_caution_about_using...,2611,- At least 13% of retail investors use AI tool...,AI-selected portfolios might perform well in a...,"[Retail Investors, Investment Tools, AI in Fin..."
2,202,Ars Technica,Google DeepMind unveils its first “thinking” r...,https://arstechnica.com/google/2025/09/google-...,"Thu, 25 Sep 2025 16:00:59 +0000",DeepMind researchers believe this is the dawn ...,True,success,https://arstechnica.com/google/2025/09/google-...,download/html/Google_DeepMind_unveils_its_firs...,2025-09-25T16:00:59Z,download/text/Google_DeepMind_unveils_its_firs...,2435,- Google DeepMind introduced Gemini Robotics 1...,DeepMind researchers believe this is the dawn ...,"[DeepMind, Robotics, AI Models, Agents, Comput..."
3,206,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,True,success,https://arstechnica.com/science/2025/09/deepmi...,download/html/DeepMind_s_robotic_ballet__An_AI...,None,download/text/DeepMind_s_robotic_ballet__An_AI...,2346,"- DeepMind developed RoboBallet, an AI system ...",An AI figures out how robots can get jobs done...,"[DeepMind, Manufacturing Robots, AI Coordinati..."
4,199,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,True,success,https://arstechnica.com/ai/2025/09/why-does-op...,download/html/Why_does_OpenAI_need_six_giant_d...,None,download/text/Why_does_OpenAI_need_six_giant_d...,2461,"- OpenAI, Oracle, and SoftBank are developing ...",OpenAI’s new $400 billion announcement reveals...,"[Stargate Project, AI Data Centers, AI Infrast..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,229,NewsAPI,From 4% To 20%: Women's Enrolment Goes Up Four...,https://www.ndtvprofit.com/technology/from-4-t...,2025-09-25T13:19:16Z,None,True,success,https://www.ndtvprofit.com/technology/from-4-t...,download/html/From_4__To_20___Women_s_Enrolmen...,2025-09-25T13:19:16Z,download/text/From_4__To_20___Women_s_Enrolmen...,2004,- Women's enrolment in AI and machine learning...,Up to 70% of women candidates who enrolled in ...,"[Women in Tech, AI Education, India, STEM Enro..."
319,22,NewsAPI,I met Sam Altman in Texas. He’s turning the ra...,https://finance.yahoo.com/news/met-sam-altman-...,2025-09-25T16:29:49Z,None,True,success,https://finance.yahoo.com/news/met-sam-altman-...,download/html/I_met_Sam_Altman_in_Texas._He_s_...,2025-09-25T16:29:49Z,download/text/I_met_Sam_Altman_in_Texas._He_s_...,9182,"- OpenAI, led by Sam Altman, is undertaking th...","In Abilene, I saw how tech companies like Open...","[AI Infrastructure, Data Centers, Compute Capa..."
320,74,NewsAPI,Google’s Data Commons MCP Server Anchors AI in...,http://www.pymnts.com/news/artificial-intellig...,2025-09-25T16:00:29

In [8]:
from do_rating import *

In [9]:
rating_df = state.headline_df.copy().fillna({
        'article_len': 1,
        'reputation': 0,
        'on_topic': 0,
        'importance': 0,
        'low_quality': 0,
    })

In [10]:
    # Ensure 'title' and 'summary' are always strings
    rating_df['title'] = rating_df['title'].fillna("")
    rating_df['title'] = rating_df['title'].astype(str)
    rating_df['summary'] = rating_df['summary'].astype(str)
    rating_df['summary'] = rating_df['summary'].fillna("")


In [23]:
    rating_df['input_text'] = rating_df['title'] + "\n" + rating_df['summary']


In [12]:
    yesterday = (datetime.now(timezone.utc)
                 - timedelta(days=1)).strftime("%Y-%m-%dT%H:%M:%SZ")
    rating_df['last_updated'] = rating_df['last_updated'].fillna(yesterday)
    rating_df["age"] = (datetime.now(timezone.utc) -
                        pd.to_datetime(rating_df['last_updated']))
    rating_df["age"] = rating_df["age"].dt.total_seconds() / (24 * 60 * 60)
    rating_df["age"] = rating_df["age"].clip(lower=0)  # no negative dates
    # only consider articles from the last week
    rating_df = rating_df[rating_df["age"] < 7].copy()
    k = np.log(2)  # 1/2 after 1 day
    # 1 point at age 0, 0 at age 1, -0.5 at age 2, -1 at age infinity
    rating_df["recency_score"] = 2 * np.exp(-k * rating_df["age"]) - 1


In [16]:
print(rating_df.loc[9].input_str)


AI medical tools found to downplay symptoms of women, ethnic minorities
- Research from MIT and the London School of Economics finds that large language model-based AI medical tools frequently downplay symptoms in female, Black, and Asian patients, leading to biased and less empathetic care recommendations.
- These AI models, including OpenAI's GPT-4, Meta's Llama 3, and Google's Gemma, have been shown to suggest lower levels of care for women and less compassionate guidance for minority groups, potentially worsening existing disparities in healthcare outcomes.
- Despite AI's growing adoption by major tech companies and health systems to aid overburdened physicians and improve diagnostics, experts warn that such biases could reinforce under-treatment patterns unless addressed by future development and deployment practices.


In [17]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_quality")


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_quality' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_quality': model=gpt-4.1, system_len=1849, user_len=246


In [18]:
class StoryRating(BaseModel):
    """StoryRating class for generic structured output rating"""
    id: int = Field(description="The id of the story")
    rating: int = Field(description="An integer rating of the story")


class StoryRatings(BaseModel):
    """StoryRatings class for structured output filtering of a list of Story"""
    items: List[StoryRating] = Field(description="List of StoryRating")


        

In [19]:

        quality_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )
    

In [24]:

        rating_df['low_quality'] = await quality_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='low_quality',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )

In [27]:
rating_df.loc[rating_df["low_quality"]>0].sort_values("low_quality")


,id,source,title,url,published,rss_summary,isAI,status,final_url,html_path,...,text_path,content_length,summary,description,topics,input_str,age,recency_score,input_text,low_quality
92,32,Feedly AI,BetterArtificial Intelligence(AI) Stock: Nebiu...,https://www.nasdaq.com/articles/better-artific...,None,None,True,success,https://www.nasdaq.com/articles/better-artific...,download/html/BetterArtificial_Intelligence_AI...,...,download/text/BetterArtificial_Intelligence_AI...,7829,- Nebius and CoreWeave have experienced extrao...,Key PointsThe demand for cloud AI infrastructu...,"[Cloud AI Infrastructure, CoreWeave, Microsoft...",BetterArtificial Intelligence(AI) Stock: Nebiu...,1.000008,-0.000005,BetterArtificial Intelligence(AI) Stock: Nebiu...,6.041735e-14
317,269,NewsAPI,"Love, lies, and algorithms: Is AI really helpi...",https://biztoc.com/x/aeeb14c7b5385560,2025-09-25T15:37:55Z,None,True,success,https://biztoc.com/x/aeeb14c7b5385560,download/html/Love__lies__and_algorithms__Is_A...,...,download/text/Love__lies__and_algorithms__Is_A...,1276,- AI is increasingly integrated into dating ap...,From dating apps and AI-powered matchmaking to...,"[Dating Apps, Matchmaking, Personal Relationsh...","Love, lies, and algorithms: Is AI really helpi...",1.471756,-0.278913,"Love, lies, and algorithms: Is AI really helpi...",8.258073e-14
195,91,The Register,Microsoft insists Copilot+ PCs are 'empowering...,https://go.theregister.com/feed/www.theregiste...,2025-09-19T16:02:07.00Z,<h4>Latest marketing blitz for a solution seek...,True,success,https://go.theregister.com/feed/www.theregiste...,download/html/Microsoft_insists_Copilot__PCs_a...,...,download/text/Microsoft_insists_Copilot__PCs_a...,4555,- Microsoft promotes Copilot+ PCs and Windows ...,Comment: Latest marketing blitz for a solution...,"[Copilot PCs, Windows on Arm, Microsoft, PC Ha...",Microsoft insists Copilot+ PCs are 'empowering...,1.000008,-0.000005,Microsoft insists Copilot+ PCs are 'empowering...,8.258073e-14
129,142,HackerNoon,"Thirty Reports, Zero News: The AI PR Machine H...",https://hackernoon.com/thirty-reports-zero-new...,"Fri, 26 Sep 2025 05:10:27 GMT",The obsession with AI as a headline in itself ...,True,success,https://hackernoon.com/thirty-reports-zero-new...,download/html/Thirty_Reports__Zero_News__The_A...,...,download/text/Thirty_Reports__Zero_News__The_A...,3458,- Over thirty AI reports from PR agencies were...,"AI studies, surveys, forecasts, and whitepaper...","[AI Journalism, PR Agencies, Media Coverage, D...","Thirty Reports, Zero News: The AI PR Machine H...",0.374174,0.543094,"Thirty Reports, Zero News: The AI PR Machine H...",1.361526e-13
145,80,HackerNoon,My First Python Web App—Built in a Weekend (Wi...,https://hackernoon.com/my-first-python-web-app...,"Tue, 23 Sep 2025 06:28:17 GMT",Having an AI explain patterns and answer quest...,True,success,https://hackernoon.com/my-first-python-web-app...,download/html/My_First_Python_Web_App_Built_in...,...,download/text/My_First_Python_Web_App_Built_in...,4886,- Developer with no prior Python web experienc...,Having an AI explain patterns and answer quest...,"[AI Assistance, Coding Assistants, FastAPI, OA...",My First Python Web App—Built in a Weekend (Wi...,1.000008,-0.000005,My First Python Web App—Built in a Weekend (Wi...,1.981009e-13
89,278,Feedly AI,Nvidia's $100B OpenAI investment fuels AI bubb...,https://www.axios.com/2025/09/25/nvidia-openai...,None,None,True,success,https://www.axios.com/2025/09/25/nvidia-openai...,download/html/Nvidia_s__100B_OpenAI_investment...,...,download/text/Nvidia_s__100B_OpenAI_investment...,1214,- Nvidia's $100 billion investment in OpenAI h...,Wall Street is already concerned about an AI b...,"[Nvidia, OpenAI, Funding, Market Bubble, AI In...",Nvidia's $100B OpenAI investment fuels AI bubb...,1.000008,-0.000005,Nvidia's $100B OpenAI investment fuels AI bubb...,1.388794e-11
105,65,Feedly AI,I tried this Google Gemini feature — it's a ch...,https://www.tomsguide.com/ai/google-

In [28]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_on_topic")


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_on_topic' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_on_topic': model=gpt-4.1, system_len=1780, user_len=240


In [29]:

        topic_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )

        rating_df['on_topic'] = await topic_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='on_topic',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )
        

In [30]:
rating_df


,id,source,title,url,published,rss_summary,isAI,status,final_url,html_path,...,content_length,summary,description,topics,input_str,age,recency_score,input_text,low_quality,on_topic
0,191,Ars Technica,ChatGPT Pulse delivers morning updates based o...,https://arstechnica.com/ai/2025/09/chatgpt-pul...,"Thu, 25 Sep 2025 20:30:52 +0000",New mobile chatbot feature analyzes conversati...,True,success,https://arstechnica.com/ai/2025/09/chatgpt-pul...,download/html/ChatGPT_Pulse_delivers_morning_u...,...,2521,"- OpenAI launched ChatGPT Pulse, a new mobile ...",New mobile chatbot feature analyzes conversati...,"[ChatGPT Pulse, Personalization, Asynchronous ...",ChatGPT Pulse delivers morning updates based o...,1.262820,-0.166543,ChatGPT Pulse delivers morning updates based o...,0.0,1.000000e+00
1,182,Ars Technica,Experts urge caution about using ChatGPT to pi...,https://arstechnica.com/information-technology...,"Thu, 25 Sep 2025 18:10:50 +0000",AI-selected portfolios might perform well in a...,True,success,https://arstechnica.com/information-technology...,download/html/Experts_urge_caution_about_using...,...,2611,- At least 13% of retail investors use AI tool...,AI-selected portfolios might perform well in a...,"[Retail Investors, Investment Tools, AI in Fin...",Experts urge caution about using ChatGPT to pi...,1.360066,-0.220871,Experts urge caution about using ChatGPT to pi...,0.0,9.999687e-01
2,202,Ars Technica,Google DeepMind unveils its first “thinking” r...,https://arstechnica.com/google/2025/09/google-...,"Thu, 25 Sep 2025 16:00:59 +0000",DeepMind researchers believe this is the dawn ...,True,success,https://arstechnica.com/google/2025/09/google-...,download/html/Google_DeepMind_unveils_its_firs...,...,2435,- Google DeepMind introduced Gemini Robotics 1...,DeepMind researchers believe this is the dawn ...,"[DeepMind, Robotics, AI Models, Agents, Comput...",Google DeepMind unveils its first “thinking” r...,1.450239,-0.268079,Google DeepMind unveils its first “thinking” r...,0.0,1.000000e+00
3,206,Ars Technica,DeepMind’s robotic ballet: An AI for coordinat...,https://arstechnica.com/science/2025/09/deepmi...,"Thu, 25 Sep 2025 11:15:40 +0000",An AI figures out how robots can get jobs done...,True,success,https://arstechnica.com/science/2025/09/deepmi...,download/html/DeepMind_s_robotic_ballet__An_AI...,...,2346,"- DeepMind developed RoboBallet, an AI system ...",An AI figures out how robots can get jobs done...,"[DeepMind, Manufacturing Robots, AI Coordinati...",DeepMind’s robotic ballet: An AI for coordinat...,1.000008,-0.000005,DeepMind’s robotic ballet: An AI for coordinat...,0.0,1.000000e+00
4,199,Ars Technica,Why does OpenAI need six giant data centers?,https://arstechnica.com/ai/2025/09/why-does-op...,"Wed, 24 Sep 2025 16:06:03 +0000",OpenAI's new $400 billion announcement reveals...,True,success,https://arstechnica.com/ai/2025/09/why-does-op...,download/html/Why_does_OpenAI_need_six_giant_d...,...,2461,"- OpenAI, Oracle, and SoftBank are developing ...",OpenAI’s new $400 billion announcement reveals...,"[Stargate Project, AI Data Centers, AI Infrast...",Why does OpenAI need six giant data centers?\n...,1.000008,-0.000005,Why does OpenAI need six giant data centers?\n...,0.0,1.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,229,NewsAPI,From 4% To 20%: Women's Enrolment Goes Up Four...,https://www.ndtvprofit.com/technology/from-4-t...,2025-09-25T13:19:16Z,None,True,success,https://www.ndtvprofit.com/technology/from-4-t...,download/html/From_4__To_20___Women_s_Enrolmen...,...,2004,- Women's enrolment in AI and machine learning...,Up to 70% of women candidates who enrolled in ...,"[Women in Tech, AI Education, India, STEM Enro...",From 4% To 20%: Women's Enrolment Goes Up Four...,1.562543,-0.322892,From 4% To 20%: Women's Enrolment Goes Up Four...,0.0,9.610239e-05
319,22,NewsAPI,I met Sam Altman in Texas. He’s turning the ra...,https://finance.yahoo.com/news/met-sam-altman-..

In [31]:
        system, user, model = LangfuseClient().get_prompt("newsagent/rate_importance")


INFO:llm:Initialized LangfuseClient
INFO:llm:Successfully retrieved prompt 'newsagent/rate_importance' from Langfuse
INFO:llm:Parsed prompt 'newsagent/rate_importance': model=gpt-4.1, system_len=2145, user_len=252


In [32]:

        importance_agent = LLMagent(
            system_prompt=system,
            user_prompt=user,
            output_type=StoryRatings,
            model=model,
            verbose=False,
            logger=logger
        )
    

In [33]:

        rating_df['important'] = await importance_agent.filter_dataframe(
            rating_df[['id', 'input_text']],
            value_field='important',
            item_list_field='results_list',
            item_id_field='id',
            chunk_size=25,
            return_probabilities=True
        )
    

In [34]:
    rating_df['bt_z'] = 0.0


In [35]:
    rating_df['adjusted_len'] = np.log10(rating_df['content_length']) - 3
    rating_df['adjusted_len'] = rating_df['adjusted_len'].clip(
        lower=0, upper=2)


/opt/anaconda3/envs/asdk/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [38]:
    rating_df['rating'] = rating_df['adjusted_len'] \
        + rating_df['on_topic'] \
        + rating_df['important'] \
        - rating_df['low_quality'] \
        + rating_df['bt_z'] \
        + rating_df['recency_score']

In [ ]:
todo: reputation 
bradley_terry